# 맛보기) 손글씨 숫자 인식

In [12]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from PIL import Image
from mnist import load_mnist

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()
    
(x_train, t_train), (x_test, t_test) = \
    load_mnist(flatten=True, normalize=False)

img = x_train[0]
label = t_train[0]
print(label)
print(img.shape)
img = img.reshape(28,28)
img_show(img)


5
(784,)


# 신경망의 추론(forward) 처리

In [20]:
import pickle 

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

def init_weight():
    
    # 여기서는 학습된 가중치 매개변수를 읽어와 추론만 가능
    
    with open("sample_weight.pkl", 'rb') as f :
        network = pickle.load(f)
    return network

def sigmoid(x):
    return 1/(1+np.exp(-x))

def softmax(x):
    c=np.max(x)
    exp_a = np.exp(x-c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

# predict가 forward

def predict(network,x):
    W1,W2,W3 = network['W1'], network['W2'], network['W3']
    b1,b2,b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x,W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1,W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2,W3) + b3
    y = softmax(a3)
    
    return y



# p = np.argmax(y) # 확률이 가장 높은 원소의 인덱스

In [29]:
x,t = get_data()
network = init_weight()

accuracy_cnt = 0

for i in range(len(x)):
    y = predict(network,x[i])
    #print(y)
    p = np.argmax(y) # 확률이 가장 높은 원소의 인덱스
    
    if p == t[i]: # p값이 라벨과 일치하면 accuracy_cnt 증가
        accuracy_cnt += 1 
        
print("Accuracy : ", str(float(accuracy_cnt) / len(x)))

Accuracy :  0.9352


In [25]:
print(len(x))

10000


# 배치 처리

#  1) 이미지 데이터 1장만 입력했을때 처리 흐름

In [30]:
x, _ = get_data()
network = init_weight()
W1, W2, W3 =  network['W1'], network['W2'], network['W3']

x.shape

(10000, 784)

In [31]:
x[0].shape

(784,)

In [32]:
W1.shape

(784, 50)

In [34]:
W2.shape

(50, 100)

In [35]:
W3.shape

(100, 10)

# 2) 이미지 데이터 100장 묶어서 입력했을때 처리 흐름 

In [47]:
x,t = get_data()
network = init_weight()

# 배치 사이즈 지정
batch_size = 100

accuracy_cnt = 0

# for i in range(len(x)):
#     y = predict(network,x[i])
#     #print(y)
#     p = np.argmax(y) # 확률이 가장 높은 원소의 인덱스
    
#     if p == t[i]: # p값이 라벨과 일치하면 accuracy_cnt 증가
#         accuracy_cnt += 1 

for i in range(0,len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

        
print("Accuracy : ", str(float(accuracy_cnt) / len(x)))

Accuracy :  0.9352


## np.argmax (x, axis=1)   ,   axis=1 : 열에서 max

In [43]:
x = np.array([[0.1, 0.8, 0.1],[0.3,0.1, 0.6], [0.2,0.5, 0.3], [0.8,0.1,0.1]])
y1 = np.argmax(x)
print(y1)

1


In [44]:
# axis=1 이 열에서 가장 큰 값 찾기

y2 = np.argmax(x, axis=1)
print(y2)

[1 2 1 0]


In [45]:
y3 = np.argmax(x, axis=0)
print(y3)

[3 0 1]


## np.sum( y == t) , true의 갯수

In [48]:
y = np.array([1,2,1,0])
t = np.array([1,2,0,0])

k = np.sum(y==t)

In [49]:
print(k)

3
